# Gallery of Large Galaxies for DR5

The purpose of this notebook is to build the gallery of large galaxies for the fifth Legacy Survey data release, DR5.

The parent sample is a diameter-limited (*D25>5* arcsec) sample defined and documented as part of the [Legacy Survey Large Galaxy Atlas](https://github.com/moustakas/LSLGA).  Here, we use the group catalog constructed in [this notebook](https://github.com/moustakas/LSLGA/blob/master/doc/nb/lslga-groups.ipynb) which used a one arcminute linking length.

In addition to requiring each galaxy (or galaxy group) to be in the DR5 footprint with 3-band *grz* imaging, we also require no more than 5% of the pixels in the image to be masked.

### Imports and paths

In [1]:
import os
import subprocess
import numpy as np

import fitsio
import matplotlib.pyplot as plt
from astropy.table import Table, Column, vstack
from PIL import Image, ImageDraw, ImageFont

from astrometry.util.util import Tan
from astrometry.util.fits import fits_table, merge_tables
from legacypipe.survey import ccds_touching_wcs, LegacySurveyData

In [2]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2
nproc = 1 # hack because multiprocessing doesn't seem to be working in jupyter-dev

In [3]:
%matplotlib inline

### Preliminaries

Define the data release and specify the desired angular radius range (in arcmin) for the galaxies (and galaxy groups) that will end up in the gallery.

In [4]:
dr = 'dr5'
if dr == 'dr5':
    PIXSCALE = 0.262
else:
    raise NotImplementedError

In [5]:
suffix = '0.05'

In [6]:
mindiameter, maxdiameter, d25min = 1.0, 10.0, 0.4 # [arcmin]
minnmembers = 3

In [7]:
GALAXY_DIAMFACTOR = 2.0
GROUP_DIAMFACTOR = 2.0

In [8]:
gallerydir = os.path.join( os.getenv('SCRATCH'), dr, 'gallery' )
galleryfile = os.path.join(gallerydir, 'gallery-{}.fits'.format(dr))

### Instantiate the LegacySurveyData class and read the full set of CCDs.

In [9]:
survey = LegacySurveyData()

In [10]:
bricks = survey.get_bricks()
allccds = survey.get_annotated_ccds()
cut = survey.photometric_ccds(allccds)
if cut is not None:
    allccds.cut(cut)
cut = survey.ccd_cuts(allccds)
allccds.cut(cut == 0)
print('Read {} CCDs.'.format(len(allccds)))

Converted brickname from |S8 to <U8
Reading annotated CCDs from /global/cscratch1/sd/desiproc/dr5/ccds-annotated-run19.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S51 to <U51
Converted plver from |S6 to <U6
Got 73860 CCDs
Reading annotated CCDs from /global/cscratch1/sd/desiproc/dr5/ccds-annotated-decals.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S61 to <U61
Converted plver from |S6 to <U6
Got 493440

### Define the parent sample based on cuts to D(25).

In [11]:
def read_groupcat(gallerydir='.', mindiameter=0.0, maxdiameter=1000.0, d25min=0.5,
                  decmin=-90.0, decmax=+90.0, ramin=0.0, ramax=360.0, 
                  minnmembers=1):
    """Read the LEDA group catalog.  Look for individual galaxies or groups with 
    diameters between [mindiameter, maxdiameter] (arcminutes) but also require the
    smallest member of each group to have D(25)>d25min (arcminute).  These cuts
    ensure that we don't build up mosaics of lots of small separated galaxies -- 
    we require at least one "big" galaxy in each mosaic.
    
    In addition, there are optional cuts on (ra,dec) which are useful for testing.
    
    minnmembers is the minimum number of members in each group.
    
    """
    groupfile = os.path.join(gallerydir, 'leda-logd25-{}-groupcat.fits'.format(suffix))
    print('Reading {}'.format(groupfile))
    cat = Table.read(groupfile)

    these = np.where((cat['DIAMETER'] / 60.0 <= maxdiameter) *
                     (cat['DIAMETER'] / 60.0 >= mindiameter) *
                     (cat['D25MIN'] / 60.0 >= d25min) *
                     (cat['NMEMBERS'] >= minnmembers) *
                     (cat['DEC'] <= decmax) *
                     (cat['DEC'] >= decmin) *
                     (cat['RA'] <= ramax) *
                     (cat['RA'] >= ramin)
                     )[0]
    outcat = cat[these] 
    outcat.add_column(Column(name='BRICKNAME', dtype='S17', shape=(6,), length=len(outcat)))

    print('Found {}/{} groups with N>{} members, each with D(25)>{}, and in the diameter range={}-{} arcmin.'.format(
            len(outcat), len(cat), minnmembers, d25min, mindiameter, maxdiameter))

    return outcat

In [12]:
parent = read_groupcat(gallerydir=gallerydir, mindiameter=mindiameter, 
                       maxdiameter=maxdiameter, d25min=d25min, 
                       minnmembers=minnmembers)
parent

Reading /global/cscratch1/sd/ioannis/dr5/gallery/leda-logd25-0.05-groupcat.fits
Found 1300/713903 groups with N>3 members, each with D(25)>0.4, and in the diameter range=1.0-10.0 arcmin.


GROUPID,GALAXY,NMEMBERS,RA,DEC,DIAMETER,D25MAX,D25MIN,BRICKNAME [6]
int32,str1000,int32,float64,float64,float32,float32,float32,bytes17
611510,"NGC7805,NGC7806,PGC000111",3,0.377,31.4395333333,109.654,84.7523,32.9725,..
611520,"PGC000113,PGC671782,PGC671745",3,0.4157,-33.76655,185.913,31.4885,28.0641,..
611701,"PGC2452515,PGC2453273,PGC2453243",3,0.8103,53.8627,164.573,65.7887,25.5948,..
611744,"PGC472503,PGC129167,PGC073219",3,0.9184,-49.8610366667,246.576,39.6416,28.7178,..
611800,"PGC087701,PGC737465,PGC737031,PGC736578,PGC737450",5,0.98631,-28.806118,370.239,30.0712,25.0122,..
611872,"PGC427313,PGC143280,PGC143288",3,1.18915,-53.6730533333,239.536,26.801,25.5948,..
611909,"PGC595015,PGC595115,PGC595097",3,1.23565,-39.7075533333,151.034,47.6597,26.801,..
611921,"PGC000346,PGC073229,ESO193-017",3,1.2675,-50.8264566667,90.7272,60.0,27.4253,..
611934,"PGC773329,PGC198124,PGC773825",3,1.2831,-25.9208633333,185.915,36.9957,26.1909,..


In [13]:
if False:
    these = (parent['RA'] > 200) * (parent['RA'] < 210) * (parent['DEC'] > 5) * (parent['DEC'] < 10.0)
    parent = parent[these]
    print(np.sum(these))

### Build the parent sample of galaxies in the DR5 footprint.

In [14]:
def _uniqccds(ccds):
    '''Get the unique set of CCD files.'''
    ccdfile = []
    [ccdfile.append('{}-{}'.format(expnum, ccdname)) for expnum,
     ccdname in zip(ccds.expnum, ccds.ccdname)]
    _, indx = np.unique(ccdfile, return_index=True)
    return ccds[indx]

In [15]:
def _galwcs(gal, factor=2.0):
    '''Build a simple WCS object for a single galaxy.
    
    '''
    diam = factor*np.ceil(gal['DIAMETER']/PIXSCALE).astype('int16') # [pixels]
    galwcs = Tan(gal['RA'], gal['DEC'], diam/2+0.5, diam/2+0.5,
                 -PIXSCALE/3600.0, 0.0, 0.0, PIXSCALE/3600.0, 
                 float(diam), float(diam))
    return galwcs

In [16]:
def _build_sample_onegalaxy(args):
    """Filler function for the multiprocessing."""
    return build_sample_onegalaxy(*args)

In [17]:
def build_sample_onegalaxy(gal, verbose=False):
    """Wrapper function to find overlapping CCDs for a given galaxy.

    First generously find the nearest set of CCDs that are near the galaxy and
    then demand that there's 3-band coverage in a much smaller region centered
    on the galaxy.

    """
    #print('Working on {}...'.format(gal['GALAXY'].strip()))
    if gal['NMEMBERS'] == 1:
        galwcs = _galwcs(gal, factor=GALAXY_DIAMFACTOR)
    else:
        galwcs = _galwcs(gal, factor=GROUP_DIAMFACTOR)
    these = ccds_touching_wcs(galwcs, allccds)

    if len(these) > 0:
        ccds1 = _uniqccds( allccds[these] )

        # Is there 3-band coverage?
        galwcs_small = _galwcs(gal, factor=1.0)
        these_small = ccds_touching_wcs(galwcs_small, ccds1)
        ccds1_small = _uniqccds( ccds1[these_small] )

        if 'g' in ccds1_small.filter and 'r' in ccds1_small.filter and 'z' in ccds1_small.filter:
            if verbose:
                print('For {} found {} CCDs, RA = {:.5f}, Dec = {:.5f}, Diameter={:.4f} arcsec'.format(
                        gal['GALAXY'].strip(), len(ccds1), gal['RA'], gal['DEC'], gal['DIAMETER']))
            
            # Also get the set of bricks touching this footprint.
            rad = gal['DIAMETER'] / 3600 # [degree]
            brickindx = survey.bricks_touching_radec_box(bricks,
                                                         gal['RA']-rad, gal['RA']+rad,
                                                         gal['DEC']-rad, gal['DEC']+rad)
            if len(brickindx) == 0 or len(brickindx) > len(gal['BRICKNAME']):
                print('This should not happen!')
                import pdb ; pdb.set_trace()
            gal['BRICKNAME'][:len(brickindx)] = bricks.brickname[brickindx]

            return [gal, ccds1]

    return None

In [18]:
def build_sample():
    """Build the sample for all galaxies with the option of multiprocessing
    (although it doesn't work on jupyter-dev)."""
    sampleargs = list()
    for cc in parent:
        sampleargs.append( (cc, False) ) # the False refers to verbose=False

    if nproc > 1:
        p = multiprocessing.Pool(nproc)
        result = p.map(_build_sample_onegalaxy, sampleargs)
        p.close()
    else:
        result = list()
        for args in sampleargs:
            result.append(_build_sample_onegalaxy(args))

    # Remove non-matching galaxies and write out the sample
    result = list(filter(None, result))
    result = list(zip(*result))

    outcat = vstack(result[0])
    outccds = merge_tables(result[1])
    print('Found {}/{} galaxies and groups in the DR5 footprint.'.format(len(outcat), len(parent)))
    
    # Add a fracmasked column.
    outcat.add_column(Column(name='FRACMASKED', shape=(3,), length=len(outcat), dtype='f2'))
    
    return outcat

In [19]:
%time sample = build_sample()
sample

Found 271/1300 galaxies and groups in the DR5 footprint.
CPU times: user 28.7 s, sys: 164 ms, total: 28.9 s
Wall time: 28.9 s


GROUPID,GALAXY,NMEMBERS,RA,DEC,DIAMETER,D25MAX,D25MIN,BRICKNAME [6],FRACMASKED [3]
int32,str1000,int32,float64,float64,float32,float32,float32,bytes20,float16
614652,"NGC0127,NGC0128,NGC0130",3,7.3139,2.86903333333,189.737,189.737,42.4767,0073p027 ..,0.0 .. 0.0
615005,"PGC1036047,PGC1036361,PGC1036593",3,8.2232,-6.13836333333,201.951,53.4751,28.0641,0081m062 ..,0.0 .. 0.0
615052,"PGC1028413,PGC143577,PGC143579",3,8.3796,-6.82209333333,151.885,27.4253,24.4428,0084m067 ..,0.0 .. 0.0
615465,"PGC1101085,PGC1100823,PGC1100450",3,9.30705,-2.13501666667,141.452,37.8574,25.0122,0093m022 ..,0.0 .. 0.0
615699,"NGC0192,NGC0196,NGC0197",3,9.8195,0.889566666667,212.301,122.504,55.9953,0098p007 ..,0.0 .. 0.0
616634,"PGC842319,NGC0247B,ESO540-024,ESO540-025,PGC002798",5,11.89698,-20.456582,480.295,62.8277,31.4885,0118m205 ..,0.0 .. 0.0
616934,"PGC090503,PGC1244573,PGC1244244",3,12.65625,3.07869666667,151.249,39.6416,28.0641,0126p030 ..,0.0 .. 0.0
617056,"IC0055,PGC1328252,PGC1328284",3,12.95565,7.69844333333,258.555,47.6597,26.1909,0129p077 ..,0.0 .. 0.0
617185,"PGC173062,PGC173063,PGC173070",3,13.28545,-3.63262,196.097,47.6597,30.7717,0131m037 ..,0.0 .. 0.0


### Retrieve FITS cutouts of each galaxy in each band.

We also compute the fraction of masked pixels in each band (so we can do some quality cuts, below) and then write out the final sample.

In [20]:
fitsdir = os.path.join(gallerydir, 'fits')
if not os.path.isdir(fitsdir):
    os.mkdir(fitsdir)

In [21]:
def galaxyname(gal, html=False):
    """We want to prevent the output FITS/png filenames from arbitrarily long
    so we use the following convention:
    
    1 member - use the galaxy name
    2-3 members - join the individual galaxy names with a hyphen
    >3 members - use the groupid with format :08d
    
    """
    if gal['NMEMBERS'] > 500:
        galaxy = 'group{:08d}'.format(gal['GROUPID'])
    else:
        if html:
            galaxy = gal['GALAXY'].strip().lower().replace(',', ' ')
        else:
            galaxy = gal['GALAXY'].strip().lower().replace(',', '-')
    
    return galaxy

In [22]:
def fits_cutouts(verbose=False):
    """Grab cutouts of every galaxy and also quickly compute the fraction of 
    pixels that are masked in each image (in fracmasked).
    
    """
    for jj, gal in enumerate(sample):
        galaxy = galaxyname(gal)
        if gal['NMEMBERS'] == 1:
            size = np.ceil(GALAXY_DIAMFACTOR*gal['DIAMETER']/PIXSCALE).astype('int16') # [pixels]
        else:
            size = np.ceil(GROUP_DIAMFACTOR*gal['DIAMETER']/PIXSCALE).astype('int16') # [pixels]

        # Get a FITS cutout and then split the file into three individual bands.
        baseurl = 'http://legacysurvey.org/viewer-dev/fits-cutout/'
        fitsurl = '{}?ra={:.6f}&dec={:.6f}&pixscale={:.3f}&size={:g}&layer=decals-{}'.format(
            baseurl, gal['RA'], gal['DEC'], PIXSCALE, size, dr)
        fitsfile = os.path.join(fitsdir, '{}.fits'.format(galaxy))
        cmd = 'wget --continue -O {:s} "{:s}"' .format(fitsfile, fitsurl)
        if os.path.isfile(os.path.join(fitsdir, '{}-{}.fits'.format(galaxy, 'g'))):
            if verbose:
                print('Skipping galaxy {}'.format(galaxy))
        else:
            if verbose:
                print(cmd)
            os.system(cmd)

            img, hdr = fitsio.read(fitsfile, ext=0, header=True)
            for ii, band in enumerate(['g', 'r', 'z']):
                bandfile = os.path.join(fitsdir, '{}-{}.fits'.format(galaxy, band))
                #print('  Writing {}'.format(bandfile))
                sample['FRACMASKED'][jj][ii] = np.sum(img[ii, :, :] == 0) / img[ii, :, :].size
                fitsio.write(bandfile, img[ii, :, :], clobber=True, header=hdr)        
            #print('  Removing {}'.format(fitsfile))
            os.remove(fitsfile)

In [23]:
%time fits_cutouts(verbose=False)

CPU times: user 8.88 s, sys: 17.6 s, total: 26.5 s
Wall time: 19min 23s


In [24]:
if os.path.isfile(galleryfile):
    os.remove(galleryfile)
    
print('Writing {}'.format(galleryfile))
sample.write(galleryfile)

Writing /global/cscratch1/sd/ioannis/dr5/gallery/gallery-dr5.fits


### Build a color image using trilogy.py

Here we require no more than 2% of the pixels in *all* the FITS images to be masked (i.e., to have values of zero) before generating a PNG file.  Then, after the PNG image has been generated, we read it back in and add a scale bar and a galaxy label.

In [25]:
fraccut = 0.02 # [fraction]

In [26]:
pngdir = os.path.join(gallerydir, 'png')
if not os.path.isdir(pngdir):
    os.mkdir(pngdir)

In [27]:
barlen30 = np.round(30.0 / PIXSCALE).astype('int')
fonttype = os.path.join(gallerydir, 'Georgia.ttf')

In [28]:
def make_png(pngsample, verbose=False):
    """Build a png file for each galaxy / group using Trilogy. 
    
    """
    for gal in pngsample:
        galaxy = galaxyname(gal)
        
        pngfile = os.path.join(pngdir, '{}.png'.format(galaxy))
        thumbfile = os.path.join(pngdir, 'thumb-{}.png'.format(galaxy))

        paramfile = os.path.join(pngdir, '{}.in'.format(galaxy))
        with open(paramfile, 'w') as param:
            param.write('B\n')
            param.write(os.path.join(fitsdir, '{}-g.fits\n'.format(galaxy)))
            param.write('\n')
            param.write('G\n')
            param.write(os.path.join(fitsdir, '{}-r.fits\n'.format(galaxy)))
            param.write('\n')        
            param.write('R\n')
            param.write(os.path.join(fitsdir, '{}-z.fits\n'.format(galaxy)))
            param.write('\n')
            param.write('indir {}\n'.format(fitsdir))
            param.write('outname {}\n'.format(pngfile))
            param.write('noiselum 0.3\n')
            param.write('satpercent 0.001\n')
            param.write('samplesize 1000\n')
            param.write('stampsize 5000\n')
            #param.write('scaling {}\n'.format(os.path.join(gallerydir, 'levels.txt')))
            param.write('colorsatfac 0.95\n')
            param.write('deletetests 1\n')
            param.write('showstamps 0\n')
            param.write('show 0\n')
            param.write('legend 0\n')
            param.write('testfirst 0\n')
        param.close()
        
        trilogy = os.path.join(os.getenv('LEGACYPIPE_DIR'), 'py', 'legacyanalysis', 'trilogy.py')
        cmd = 'python {} {}'.format(trilogy, paramfile)
        #print(cmd)
        os.system(cmd)
        for txt in ('levels.txt', 'trilogyfilterlog.txt'):
            thisfile = os.path.join(os.getenv('LEGACYPIPE_DIR'), 'doc', 'nb', txt)
            os.remove(thisfile)
        os.remove(os.path.join(pngdir, '{}_filters.txt'.format(galaxy)))
        os.remove(os.path.join(pngdir, '{}.in'.format(galaxy)))
    
        im = Image.open(pngfile)
        sz = im.size
        fntsize = np.round(sz[0]/28).astype('int')
        width = np.round(sz[0]/175).astype('int')
        font = ImageFont.truetype(fonttype, size=fntsize)
        draw = ImageDraw.Draw(im)
        # Label the galaxy name--
        draw.text((0+fntsize*2, 0+fntsize*2), galaxy.upper(), font=font)
        #draw.text((sz[0]-fntsize*6, sz[1]-fntsize*3), galaxy.upper(), font=font)
        # Add a 30 arcsec scale bar--
        x0, x1, yy = sz[1]-fntsize*2-barlen30, sz[1]-fntsize*2, sz[0]-fntsize*2
        draw.line((x0, yy, x1, yy), fill='white', width=width)
        im.save(pngfile)    
        
        # Generate a thumbnail
        cmd = '/usr/bin/convert -thumbnail 300x300 {} {}'.format(pngfile, thumbfile)
        os.system(cmd)
        
        if verbose:
            print('Wrote {}'.format(pngfile))    

In [29]:
indx = np.sum(sample['FRACMASKED'] < fraccut, axis=1) == 3
pngsample = sample[indx]
print('Identified {} groups with <2% pixels masked in grz.'.format(len(pngsample)))

Identified 174 groups with <2% pixels masked in grz.


In [30]:
pngsample

GROUPID,GALAXY,NMEMBERS,RA,DEC,DIAMETER,D25MAX,D25MIN,BRICKNAME [6],FRACMASKED [3]
int32,str1000,int32,float64,float64,float32,float32,float32,bytes20,float16
614652,"NGC0127,NGC0128,NGC0130",3,7.3139,2.86903333333,189.737,189.737,42.4767,0073p027 ..,4.7684e-07 .. 7.1526e-06
615005,"PGC1036047,PGC1036361,PGC1036593",3,8.2232,-6.13836333333,201.951,53.4751,28.0641,0081m062 ..,5.3823e-05 .. 7.7784e-05
615052,"PGC1028413,PGC143577,PGC143579",3,8.3796,-6.82209333333,151.885,27.4253,24.4428,0084m067 ..,0.0 .. 0.0
615465,"PGC1101085,PGC1100823,PGC1100450",3,9.30705,-2.13501666667,141.452,37.8574,25.0122,0093m022 ..,1.2875e-05 .. 9.4175e-06
616934,"PGC090503,PGC1244573,PGC1244244",3,12.65625,3.07869666667,151.249,39.6416,28.0641,0126p030 ..,0.0 .. 0.0
617185,"PGC173062,PGC173063,PGC173070",3,13.28545,-3.63262,196.097,47.6597,30.7717,0131m037 ..,9.6858e-05 .. 0.0
618099,"PGC003597,PGC914550,PGC173260",3,15.12245,-15.18764,219.216,48.7698,25.0122,0151m152 ..,0.00061321 .. 0.0
619319,"PGC929534,PGC929768,PGC004283",3,17.8845,-14.0488366667,185.516,45.5147,26.1909,0178m140 ..,0.0010471 .. 6.1274e-05
619432,"PGC1036232,PGC1036166,PGC144001,PGC1036077",4,18.1082625,-6.1616275,288.214,54.7206,24.4428,0179m062 ..,0.0 .. 1.7881e-07


In [31]:
%time make_png(pngsample, verbose=False)

CPU times: user 2min 15s, sys: 10 s, total: 2min 25s
Wall time: 12min 42s


### Finally, assemble the webpage of good and rejected gallery images.

To test the webpage before release, do
  * rsync -auvP /global/cscratch1/sd/ioannis/dr5/gallery/png /global/project/projectdirs/cosmo/www/temp/ioannis/dr5/gallery/
  * rsync -auvP /global/cscratch1/sd/ioannis/dr5/gallery/index.html /global/project/projectdirs/cosmo/www/temp/ioannis/dr5/gallery/

In [32]:
reject = ['AllGood']
#reject = ['IC3155']
keep = np.zeros(len(pngsample), dtype=bool)
for ii, gg in enumerate(pngsample['GALAXY']):
    for rej in np.atleast_1d(reject):
        keep[ii] = rej not in gg
pngkeep = pngsample[keep]
if np.sum(~keep) > 0:
    pngrej = pngsample[~keep]

In [33]:
nperrow = 5
nrow = np.ceil(len(pngkeep) / nperrow).astype('int')
pngsplit = np.array_split(pngkeep, nrow)
print('Splitting the sample into {} rows with {} mosaics per row.'.format(nrow, nperrow))

Splitting the sample into 35 rows with 5 mosaics per row.


In [34]:
htmlfile = os.path.join(gallerydir, 'index.html')
baseurl = 'http://legacysurvey.org/viewer-dev'

In [35]:
with open(htmlfile, 'w') as html:
    html.write('<html><head>\n')
    html.write('<style type="text/css">\n')
    html.write('table {table-layout: fixed; width: 100%;}\n')
    html.write('td {word-wrap: break-word;}\n')
    html.write('p {width: 100%;}\n')
    html.write('</style>\n')
    html.write('</head><body>\n')
    html.write("""<p>This gallery highlights some of the galaxy groups in the DR5 footprint.  The
    groups were identified using an input catalog of galaxies with angular diameters D(25)>24 arcsec
    and a friends-of-friends algorithm with a linking length of 2.5 arcminute.  The color mosaics
    were generated using <a class="reference external" href="http://www.stsci.edu/~dcoe/trilogy">trilogy.py</a>.  
    Each thumbnail links to a larger image while the galaxy names below the thumbnails link to the
    <a href="http://legacysurvey.org/viewer">Sky Viewer</a>.  For reference, the horizontal white bar in 
    each image represents 30 arcsec.</p>\n""")
    html.write('<table><tbody>\n')
    for pngrow in pngsplit:
        html.write('<tr>\n')
        for gal in pngrow:
            galaxy = galaxyname(gal)
            pngfile = os.path.join('png', '{}.png'.format(galaxy))
            thumbfile = os.path.join('png', 'thumb-{}.png'.format(galaxy))
            img = 'align="center" valign="center" src="{}" alt="{}"'.format(thumbfile, galaxy.upper())
            html.write('<td width="15%"><a href="{}"><img {} /></a></td>\n'.format(pngfile, img))
        html.write('</tr><tr>\n')
        for gal in pngrow:
            galaxy = galaxyname(gal, html=True)
            href = '{}/?layer=decals-{}&ra={:.8f}&dec={:.8f}&zoom=13'.format(baseurl, dr, gal['RA'], gal['DEC'])
            html.write('<td width="15%"><a href="{}">{}</a></td>\n'.format(href, galaxy.upper()))
        html.write('</tr>\n')
    html.write('</table></tbody>')
    html.write('</body></html>\n')

In [36]:
htmlfile_reject = os.path.join(gallerydir, 'index-reject.html')

In [37]:
if len(pngrej) > 0:
    with open(htmlfile_reject, 'w') as html:
        html.write('<html><body>\n')
        for gal in pngrej:
            galaxy = galaxyname(gal)
            pngfile = os.path.join('png', '{}.png'.format(galaxy))
            img = 'src="{}" alt="{}" width="500px"'.format(pngfile, galaxy.upper())
            html.write('<a href="{}/?ra={:.8f}&dec={:.8f}&layer=decals-{}"><img {}></a>\n'.format(
                    baseurl, gal['RA'], gal['DEC'], dr, img))
        html.write('</body></html>\n')

NameError: name 'pngrej' is not defined